In [1]:
!pip install --upgrade --user --quiet google-cloud-aiplatform

In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [2]:
PROJECT_ID = "angelic-bee-193823"  
LOCATION = "us-central1"  

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [1]:
!gcloud config list

[compute]
region = us-central1
[core]
account = marvinngesa@gmail.com
disable_usage_reporting = True
project = angelic-bee-193823
[dataproc]
region = us-central1

Your active configuration is: [default]


In [3]:
import requests
from vertexai.generative_models import (
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

#### Chat example: Using Function Calling in a chat session to answer user's questions about the Google Store

In this example, you'll use Function Calling along with the chat modality in the Gemini model to help customers get information about products in the Google Store.

You'll start by defining three functions: one to get product information, another to get the location of the closest stores, and one more to place an order:

In [4]:
get_product_info = FunctionDeclaration(
    name="get_product_info",
    description="Get the stock amount and identifier for a given product",
    parameters={
        "type": "object",
        "properties": {
            "product_name": {"type": "string", "description": "Product name"}
        },
    },
)

get_store_location = FunctionDeclaration(
    name="get_store_location",
    description="Get the location of the closest store",
    parameters={
        "type": "object",
        "properties": {"location": {"type": "string", "description": "Location"}},
    },
)

place_order = FunctionDeclaration(
    name="place_order",
    description="Place an order",
    parameters={
        "type": "object",
        "properties": {
            "product": {"type": "string", "description": "Product name"},
            "address": {"type": "string", "description": "Shipping address"},
        },
    },
)

Note that function parameters are specified as a Python dictionary in accordance with the OpenAPI JSON schema format.

Define a tool that allows the Gemini model to select from the set of 3 functions:

In [6]:
model = GenerativeModel(
    "gemini-1.5-pro",
    generation_config=GenerationConfig(temperature=0),
    tools=[retail_tool],
)
chat = model.start_chat()

In [5]:
retail_tool = Tool(
    function_declarations=[
        get_product_info,
        get_store_location,
        place_order,
    ],
)
     

Now you can initialize the Gemini model with Function Calling in a multi-turn chat session.

You can specify the tools kwarg when initializing the model to avoid having to send this kwarg with every subsequent request:

Note: The temperature parameter controls the degree of randomness in this generation. Lower temperatures are good for functions that require deterministic parameter values, while higher temperatures are good for functions with parameters that accept more diverse or creative parameter values. A temperature of 0 is deterministic. In this case, responses for a given prompt are mostly deterministic, but a small amount of variation is still possible.

We're ready to chat! Let's start the conversation by asking if a certain product is in stock:

In [7]:
prompt = """
Do you have the Pixel 8 Pro in stock?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_product_info"
  args {
    fields {
      key: "product_name"
      value {
        string_value: "Pixel 8 Pro"
      }
    }
  }
}

The response from the Gemini API consists of a structured data object that contains the name and parameters of the function that Gemini selected out of the available functions.

Since this notebook focuses on the ability to extract function parameters and generate function calls, you'll use mock data to feed synthetic responses back to the Gemini model rather than sending a request to an API server (not to worry, we'll make an actual API call in a later example!):

In [8]:
# Here you can use your preferred method to make an API request and get a response.
# In this example, we'll use synthetic data to simulate a payload from an external API response.

api_response = {"sku": "GA04834-US", "in_stock": "yes"}

In reality, you would execute function calls against an external system or database using your desired client library or REST API.

Now, you can pass the response from the (mock) API request and generate a response for the end user:

In [9]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_info",
        response={
            "content": api_response,
        },
    ),
)
response.text

'Yes, the Pixel 8 Pro is in stock. \n'

Next, the user might ask where they can buy a different phone from a nearby store:

In [10]:
prompt = """
What about the Pixel 8? Is there a store in
Mountain View, CA that I can visit to try one out?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_product_info"
  args {
    fields {
      key: "product_name"
      value {
        string_value: "Pixel 8"
      }
    }
  }
}

Again, you get a response with structured data, and the Gemini model selected the get_product_info function. This happened since the user asked about the "Pixel 8" phone this time rather than the "Pixel 8 Pro" phone.

Now you can build another synthetic payload that would come from an external API:

In [11]:
# Here you can use your preferred method to make an API request and get a response.
# In this example, we'll use synthetic data to simulate a payload from an external API response.

api_response = {"sku": "GA08475-US", "in_stock": "yes"}

Again, you can pass the response from the (mock) API request back to the Gemini model:

In [12]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_info",
        response={
            "content": api_response,
        },
    ),
)
response.candidates[0].content.parts[0]

function_call {
  name: "get_store_location"
  args {
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
  }
}

Wait a minute! Why did the Gemini API respond with a second function call to get_store_location this time rather than a natural language summary? Look closely at the prompt that you used in this conversation turn a few cells up, and you'll notice that the user asked about a product -and- the location of a store.

In cases like this when two or more functions are defined (or when the model predicts multiple function calls to the same function), the Gemini model might sometimes return back-to-back or parallel function call responses within a single conversation turn.

This is expected behavior since the Gemini model predicts which functions it should call at runtime, what order it should call dependent functions in, and which function calls can be parallelized, so that the model can gather enough information to generate a natural language response.

Not to worry! You can repeat the same steps as before and build another synthetic payload that would come from an external API:

In [13]:
# Here you can use your preferred method to make an API request and get a response.
# In this example, we'll use synthetic data to simulate a payload from an external API response.

api_response = {"store": "2000 N Shoreline Blvd, Mountain View, CA 94043, US"}

And you can pass the response from the (mock) API request back to the Gemini model:

In [14]:
response = chat.send_message(
    Part.from_function_response(
        name="get_store_location",
        response={
            "content": api_response,
        },
    ),
)
response.text
     

'Yes, the Pixel 8 is in stock. You can visit the store at 2000 N Shoreline Blvd, Mountain View, CA 94043, US to try one out. \n'

Nice work!

Within a single conversation turn, the Gemini model requested 2 function calls in a row before returning a natural language summary. In reality, you might follow this pattern if you need to make an API call to an inventory management system, and another call to a store location database, customer management system, or document repository.

Finally, the user might ask to order a phone and have it shipped to their address:

In [15]:
prompt = """
I'd like to order a Pixel 8 Pro and have it shipped to 1155 Borregas Ave, Sunnyvale, CA 94089.
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "place_order"
  args {
    fields {
      key: "address"
      value {
        string_value: "1155 Borregas Ave, Sunnyvale, CA 94089"
      }
    }
    fields {
      key: "product"
      value {
        string_value: "Pixel 8 Pro"
      }
    }
  }
}

Perfect! The Gemini model extracted the user's selected product and their address. Now you can call an API to place the order:

In [16]:
# This is where you would make an API request to return the status of their order.
# Use synthetic data to simulate a response payload from an external API.

api_response = {
    "payment_status": "paid",
    "order_number": 12345,
    "est_arrival": "2 days",
}

And send the payload from the external API call so that the Gemini API returns a natural language summary to the end user.

In [17]:
response = chat.send_message(
    Part.from_function_response(
        name="place_order",
        response={
            "content": api_response,
        },
    ),
)
response.text

'Your order has been placed and will arrive within 2 days. Your order number is 12345. \n'

And you're done!

You were able to have a multi-turn conversation with the Gemini model using function calls, handling payloads, and generating natural language summaries that incorporated the information from the external systems.